# Loading data and preprocessing

In [1]:
%matplotlib inline

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
plt.style.use("seaborn")

In [2]:
from sklearn.datasets import load_boston
boston = load_boston()

In [3]:
data = pd.DataFrame(data=boston.data, columns=boston.feature_names)
data["MEDV"] = boston.target
data.head()

,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,B,LSTAT,MEDV
0,0.00632,18.0,2.31,0.0,0.538,6.575,65.2,4.0900,1.0,296.0,15.3,396.90,4.98,24.0
1,0.02731,0.0,7.07,0.0,0.469,6.421,78.9,4.9671,2.0,242.0,17.8,396.90,9.14,21.6
2,0.02729,0.0,7.07,0.0,0.469,7.185,61.1,4.9671,2.0,242.0,17.8,392.83,4.03,34.7
3,0.03237,0.0,2.18,0.0,0.458,6.998,45.8,6.0622,3.0,222.0,18.7,394.63,2.94,33.4
4,0.06905,0.0,2.18,0.0,0.458,7.147,54.2,6.0622,3.0,222.0,18.7,396.90,5.33,36.2


In [4]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score
from sklearn.linear_model import SGDRegressor
from sklearn.preprocessing import StandardScaler
from tqdm.notebook import tqdm

In [5]:
X = data[["RM"]]
y = data["MEDV"]

In [6]:
X_train, X_test, Y_train, Y_test = train_test_split(X, y, test_size=0.33, random_state=18)

In [7]:
X_filtered = data[(data["MEDV"] < 50)][["RM"]]
y_filtered = data[(data["MEDV"] < 50)]["MEDV"]

In [8]:
X_train_filtered, X_test_filtered, Y_train_filtered, Y_test_filtered = train_test_split(X, y, test_size=0.33, random_state=18)

In [9]:
def z_scaler(feature):
    return (feature - np.mean(feature)) / np.std(feature)

In [10]:
def min_max(feature):
    return (feature - feature.min()) / (feature.max() - feature.min())

In [11]:
X_scaled = z_scaler(X_filtered)
y_scaled = z_scaler(y_filtered)

In [12]:
X_train_scaled, X_test_scaled, Y_train_scaled, Y_test_scaled = train_test_split(X_scaled, y_scaled, test_size=0.33, random_state=18)

In [13]:
X_minmaxed = min_max(X_filtered)
y_minmaxed = min_max(y_filtered)

In [14]:
X_train_minmaxed, X_test_minmaxed, Y_train_minmaxed, Y_test_minmaxed = train_test_split(X_minmaxed, y_minmaxed, test_size=0.33, random_state=18)

In [15]:
from sklearn.preprocessing import StandardScaler
X_train_final, X_test_final, Y_train_final, Y_test_final = train_test_split(X, y, test_size = 0.33, random_state = 18)
x_scaler = StandardScaler()
y_scaler = StandardScaler()
X_train_final = x_scaler.fit_transform(X_train_final)
Y_train_final = y_scaler.fit_transform(Y_train_final.values.reshape(-1, 1))
X_test_final = x_scaler.transform(X_test_final)
Y_test_final = y_scaler.transform(Y_test_final.values.reshape(-1, 1))

# Class definitions

###### GDRegressorOriginal, GDRegressorChanged

In [16]:
class GDRegressorOriginal:

    def __init__(self, alpha=0.01, n_iter=100, progress=True):
        self.alpha = alpha
        self.n_iter = n_iter
        self.disable = not progress
        self.coef_ = np.asarray([0.0])
        self.intercept_ = 0.0
        self.theta_history = np.zeros((n_iter + 1, n_iter + 1))
        self.loss_history = np.zeros(n_iter+1)
        self.precision = 1e-5

    def fit(self, X, y):
      # since this is for linear regression, it should be okay
      if isinstance(X, pd.Series):
        X = X.to_numpy().ravel()
      if isinstance(y, pd.Series):
        y = y.to_numpy().ravel()
      for i in tqdm(np.arange(self.n_iter), disable=self.disable):
        cur_loss = self._loss(X, y, self.coef_, self.intercept_)
        self.loss_history[i] = cur_loss
        if np.isnan(cur_loss):
          # something went wrong and the loss is nan
          return
        if np.isclose(0.0, cur_loss, self.precision):
          return # overfit detected... i guess
        coef_grad, intercept_grad = self._gradient(X, y, self.coef_, self.intercept_)
        self.coef_ -= coef_grad * self.alpha
        self.intercept_ -= intercept_grad * self.alpha
        self.theta_history[i + 1, :] = self.coef_[0] # see top comment
        self.theta_history[:, i + 1] = self.intercept_
      self.loss_history[-1] = self._loss(X, y, self.coef_, self.intercept_) # fill last position

    def predict(self, X_test):
        return self._regress(X_test, self.coef_, self.intercept_)

    def _gradient(self, X, y, coef, intercept):
      coef_grad = 2 * np.mean((self._regress(X, coef, intercept) - y) * X)
      intercept_grad = 2 * np.mean(self._regress(X, coef, intercept)- y)
      return (coef_grad, intercept_grad)

    def _loss(self, X, y, coef, intercept):
      # MSE used here.
      return np.mean(np.square(self._regress(X, coef, intercept) - y))

    def _regress(self, X, coef, intercept):
      return coef * X + intercept

In [17]:
class GDRegressorChanged:

    def __init__(self, alpha=0.01, n_iter=100, progress=True):
        self.alpha = alpha
        self.n_iter = n_iter
        self.disable = not progress
        self.coef_ = np.asarray([0.0])
        self.intercept_ = 0.0
        self.theta_history = np.zeros((n_iter + 1, n_iter + 1))
        self.loss_history = np.zeros(n_iter+1)
        self.precision = 1e-5

    def fit(self, X, y):
      # since this is for linear regression, it should be okay
      if isinstance(X, pd.Series):
        X = X.to_numpy().ravel()
      if isinstance(y, pd.Series):
        y = y.to_numpy().ravel()
      for i in tqdm(np.arange(self.n_iter), disable=self.disable):
        cur_loss = self._loss(X, y, self.coef_, self.intercept_)
        self.loss_history[i] = cur_loss
        if np.isnan(cur_loss):
          # something went wrong and the loss is nan
          return
        if np.isclose(0.0, cur_loss, self.precision):
          return # overfit detected... i guess
        coef_grad, intercept_grad = self._gradient(X, y, self.coef_, self.intercept_)
        self.coef_ -= coef_grad * self.alpha
        self.intercept_ -= intercept_grad * self.alpha
        self.theta_history[i + 1, :] = self.coef_[0] # see top comment
        self.theta_history[:, i + 1] = self.intercept_
      self.loss_history[-1] = self._loss(X, y, self.coef_, self.intercept_) # fill last position

    def predict(self, X_test):
        return self._regress(X_test, self.coef_, self.intercept_)

    def _gradient(self, X, y, coef, intercept):
      coef_grad = np.mean((self._regress(X, coef, intercept) - y) * X)
      intercept_grad = -np.mean(self._regress(X, coef, intercept)- y)
      return (coef_grad, intercept_grad)

    def _loss(self, X, y, coef, intercept):
      # MSE used here.
      return np.mean(np.square(self._regress(X, coef, intercept) - y))

    def _regress(self, X, coef, intercept):
      return coef * X + intercept

In [18]:
class GDRegressor:

    def __init__(self, alpha=0.01, n_iter=100, progress=True):
        self.alpha = alpha
        self.n_iter = n_iter
        self.disable = not progress
        self.loss_history = np.asarray([])
        self.batch_size = 1 # chosen by fair dice roll

    def fit(self, X, y):
      X = np.hstack([np.ones((X.shape[0], 1)), X])
      self.theta = np.zeros(X.shape[1])
      self.theta_history = self.theta.copy()
      for _ in tqdm(np.arange(self.n_iter), disable=self.disable):
        loss = self._cost(X, y)
        self.loss_history = np.hstack([self.loss_history, loss])
        gradient = self._gradient(X, y)
        self.theta -= self.alpha * gradient
        self.theta_history = np.vstack([self.theta_history, self.theta])
        
      self.loss_history = np.hstack([self.loss_history, self._cost(X, y)])
      self.coef_ = self.theta[1:]
      self.intercept_ = self.theta[0]

    def predict(self, X_test):
      return X_test @ self.coef_ + self.intercept_

    def _gradient(self, X, y):
      return np.dot(X.T, self._loss(X, y)) / X.shape[0]

    def _loss(self, X, y):
      return np.dot(X, self.theta) - y

    def _cost(self, X, y):
      return np.square(np.mean(self._loss(X, y)))

In [19]:
from sklearn.linear_model import SGDRegressor
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error

# Metrics

In [20]:
def root_mean_squared_error(y_hat, y):
    return np.sqrt(np.mean(np.square(y_hat - y)))

In [21]:
def determination(y_hat, y):
    numerator = ((y - y_hat) ** 2).sum(axis=0, dtype=np.float64)
    denominator = ((y - np.average(y, axis=0)) ** 2).sum(axis=0, dtype=np.float64)
    output_scores = 1 - (numerator / denominator)
    return np.average(output_scores)

# Test internals

In [22]:
ALPHA_PRECISION = 1e-2

In [23]:
tests = pd.DataFrame(columns=["Class Name", "Test", "Learning Rate", "Number of iterations", "Slope", "Intercept", "RMSE", "R-Squared", "Variance_slope", "Variance_intercept", "Variance_RMSE", "Variance_R-Squared"])

In [24]:
def guess_params(Regressor):
  X_train, X_test, Y_train, Y_test = train_test_split(X_scaled, y_scaled, test_size=0.33, random_state=18)
  min_rmse = np.double(np.iinfo(np.int32).max) # i hope that's enough?
  max_rsquared = -1.0
  optimal_alpha = 0.0
  for alpha in tqdm(np.arange(1e-5, 1, ALPHA_PRECISION)):
    model = Regressor(alpha=alpha, n_iter=300, progress=False)
    model.fit(X_train.to_numpy(), Y_train.to_numpy())
    Y_pred = model.predict(X_test.to_numpy())
    if root_mean_squared_error(Y_pred, Y_test.to_numpy()) < min_rmse and determination(Y_pred, Y_test.to_numpy()) > max_rsquared:
      min_rmse = root_mean_squared_error(Y_pred, Y_test.to_numpy())
      max_rsquared = determination(Y_pred, Y_test.to_numpy())
      optimal_alpha = alpha
    del model

  min_rmse = np.double(np.iinfo(np.int32).max) # i hope that's enough?
  max_rsquared = -1.0
  optimal_iters = 0
  for n_iter in tqdm(np.arange(1, 10000, 1000)):
    model = Regressor(alpha=optimal_alpha, n_iter=n_iter, progress=False)
    model.fit(X_train.to_numpy(), Y_train.to_numpy())
    Y_pred = model.predict(X_test.to_numpy())
    if root_mean_squared_error(Y_pred, Y_test.to_numpy()) < min_rmse and determination(Y_pred, Y_test.to_numpy()) > max_rsquared:
      min_rmse = root_mean_squared_error(Y_pred, Y_test.to_numpy())
      max_rsquared = determination(Y_pred, Y_test.to_numpy())
      optimal_iters = int(n_iter)
    del model

  if optimal_iters >= 1000:
    for n_iter in tqdm(np.arange(optimal_iters-1000, optimal_iters+1000, 100)):
      model = Regressor(alpha=optimal_alpha, n_iter=int(n_iter), progress=False)
      model.fit(X_train.to_numpy(), Y_train.to_numpy())
      Y_pred = model.predict(X_test.to_numpy())
      if root_mean_squared_error(Y_pred, Y_test.to_numpy()) < min_rmse and determination(Y_pred, Y_test.to_numpy()) > max_rsquared:
        min_rmse = root_mean_squared_error(Y_pred, Y_test.to_numpy())
        max_rsquared = determination(Y_pred, Y_test.to_numpy())
        optimal_iters = int(n_iter)
      del model
    
  if optimal_iters >= 100:
    for n_iter in tqdm(np.arange(optimal_iters-100, optimal_iters+100, 10)):
      model = Regressor(alpha=optimal_alpha, n_iter=int(n_iter), progress=False)
      model.fit(X_train.to_numpy(), Y_train.to_numpy())
      Y_pred = model.predict(X_test.to_numpy())
      if root_mean_squared_error(Y_pred, Y_test.to_numpy()) < min_rmse and determination(Y_pred, Y_test.to_numpy()) > max_rsquared:
        min_rmse = root_mean_squared_error(Y_pred, Y_test.to_numpy())
        max_rsquared = determination(Y_pred, Y_test.to_numpy())
        optimal_iters = int(n_iter)
      del model

  if optimal_iters >= 10:
    for n_iter in tqdm(np.arange(optimal_iters-10, optimal_iters+10)):
      model = Regressor(alpha=optimal_alpha, n_iter=int(n_iter), progress=False)
      model.fit(X_train.to_numpy(), Y_train.to_numpy())
      Y_pred = model.predict(X_test.to_numpy())
      if root_mean_squared_error(Y_pred, Y_test.to_numpy()) < min_rmse and determination(Y_pred, Y_test.to_numpy()) > max_rsquared:
        min_rmse = root_mean_squared_error(Y_pred, Y_test.to_numpy())
        max_rsquared = determination(Y_pred, Y_test.to_numpy())
        optimal_iters = int(n_iter)
      del model

  return optimal_alpha, optimal_iters

In [25]:
def coef_variance(slope, intercept, test):
  if test == "first":
    return 3.80680876 - slope, -1.0726158227418863 - intercept
  if test == "rmse":
    return None, None
  if test == "filter":
    return None, None
  if test == "z-scale":
    return None, None
  if test == "min-max":
    return None, None
  if test == "final":
    return 0.55 - slope, 8.65898517e-05 - intercept

def metrics_variance(rmse, rsquared, test):
  if test == "first":
    return None, None
  if test == "rmse":
    return 6.452267254184752 - rmse, 0.4957549780871502 - rsquared
  if test == "filter":
    return 5.65586397329336 - rmse, 0.5520077722064608 - rsquared
  if test == "z-scale":
    return 0.7176681352663024 - rmse, 0.553778391666462 - rsquared
  if test == "min-max":
    return None, None
  if test == "final":
    return 0.48 - rmse, 0.49 - rsquared

In [26]:
def run_first_test(Regressor):
  model = Regressor(alpha=0.02, n_iter=100)
  model.fit(X_train.to_numpy(), Y_train.to_numpy())
  variance_slope, variance_intercept = coef_variance(model.coef_, model.intercept_, "first")
  variance_rmse, variance_r_squared = metrics_variance(0, 0, "first")
  return 0.02, 100, model.coef_, model.intercept_, None, None, variance_slope, variance_intercept, variance_rmse, variance_r_squared

def run_rmse_test(Regressor):
  model = Regressor(alpha=0.02, n_iter=4000)
  model.fit(X_train.to_numpy(), Y_train.to_numpy())
  Y_pred = model.predict(X_test)
  rmse = root_mean_squared_error(Y_pred.to_numpy(), Y_test.to_numpy())
  r_squared = determination(Y_pred.to_numpy(), Y_test.to_numpy())
  variance_slope, variance_intercept = coef_variance(model.coef_, model.intercept_, "rmse")
  variance_rmse, variance_r_squared = metrics_variance(rmse, r_squared, "rmse")
  return 0.02, 4000, model.coef_, model.intercept_, rmse, r_squared, variance_slope, variance_intercept, variance_rmse, variance_r_squared

def run_filter_test(Regressor):
  model = Regressor(alpha=0.02, n_iter=4000)
  model.fit(X_train_filtered.to_numpy(), Y_train_filtered.to_numpy())
  Y_pred = model.predict(X_test_filtered)
  rmse = root_mean_squared_error(Y_pred.to_numpy(), Y_test_filtered.to_numpy())
  r_squared = determination(Y_pred.to_numpy(), Y_test_filtered.to_numpy())
  variance_slope, variance_intercept = coef_variance(model.coef_, model.intercept_, "filter")
  variance_rmse, variance_r_squared = metrics_variance(rmse, r_squared, "filter")
  return 0.02, 4000, model.coef_, model.intercept_, rmse, r_squared, variance_slope, variance_intercept, variance_rmse, variance_r_squared

def run_z_scale_test(Regressor):
  model = Regressor(alpha=0.02, n_iter=300)
  model.fit(X_train_scaled.to_numpy(), Y_train_scaled.to_numpy())
  Y_pred = model.predict(X_test_scaled)
  rmse = root_mean_squared_error(Y_pred.to_numpy(), Y_test_scaled.to_numpy())
  r_squared = determination(Y_pred.to_numpy(), Y_test_scaled.to_numpy())
  variance_slope, variance_intercept = coef_variance(model.coef_, model.intercept_, "z-scale")
  variance_rmse, variance_r_squared = metrics_variance(rmse, r_squared, "z-scale")
  return 0.02, 300, model.coef_, model.intercept_, rmse, r_squared, variance_slope, variance_intercept, variance_rmse, variance_r_squared

def run_min_max_test(Regressor):
  model = Regressor(alpha=0.02, n_iter=300)
  model.fit(X_train_minmaxed.to_numpy(), Y_train_minmaxed.to_numpy())
  Y_pred = model.predict(X_test_minmaxed)
  rmse = root_mean_squared_error(Y_pred.to_numpy(), Y_test_minmaxed.to_numpy())
  r_squared = determination(Y_pred.to_numpy(), Y_test_minmaxed.to_numpy())
  variance_slope, variance_intercept = coef_variance(model.coef_, model.intercept_, "min-max")
  variance_rmse, variance_r_squared = metrics_variance(rmse, r_squared, "min-max")
  return 0.02, 300, model.coef_, model.intercept_, rmse, r_squared, variance_slope, variance_intercept, variance_rmse, variance_r_squared

def run_final_test(Regressor):
  optimal_alpha, optimal_iters = guess_params(Regressor)
  model = Regressor(alpha=optimal_alpha, n_iter=optimal_iters)
  model.fit(X_train_final, Y_train_final.ravel())
  Y_pred = model.predict(X_test_final)
  rmse = root_mean_squared_error(Y_pred, Y_test_final)
  r_squared = determination(Y_pred, Y_test_final)
  variance_slope, variance_intercept = coef_variance(model.coef_, model.intercept_, "final")
  variance_rmse, variance_r_squared = metrics_variance(rmse, r_squared, "final")
  return optimal_alpha, optimal_iters, model.coef_, model.intercept_, rmse, r_squared, variance_slope, variance_intercept, variance_rmse, variance_r_squared

# Testing

In [27]:
%%time
my_matrices_first = np.asarray(["My implementation after changes", "first"] + list(run_first_test(GDRegressor)), dtype=object)
my_matrices_rmse = np.asarray(["My implementation after changes", "rmse"] + list(run_rmse_test(GDRegressor)), dtype=object)
my_matrices_filter = np.asarray(["My implementation after changes", "filter"] + list(run_filter_test(GDRegressor)), dtype=object)
my_matrices_z_scale = np.asarray(["My implementation after changes", "z-scale"] + list(run_z_scale_test(GDRegressor)), dtype=object)
my_matrices_min_max = np.asarray(["My implementation after changes", "min-max"] + list(run_min_max_test(GDRegressor)), dtype=object)
my_matrices_final = np.asarray(["My implementation after changes", "final"] + list(run_final_test(GDRegressor)), dtype=object)
tests.loc[tests.shape[0]] = my_matrices_first
tests.loc[tests.shape[0]] = my_matrices_rmse
tests.loc[tests.shape[0]] = my_matrices_filter
tests.loc[tests.shape[0]] = my_matrices_z_scale
tests.loc[tests.shape[0]] = my_matrices_min_max
tests.loc[tests.shape[0]] = my_matrices_final


CPU times: user 11.7 s, sys: 197 ms, total: 11.9 s
Wall time: 12.2 s


In [28]:
skl_model = SGDRegressor(loss="squared_loss", learning_rate="constant", max_iter=100, eta0=0.0001)
skl_model.fit(X_train_final, Y_train_final.ravel())
Y_pred = skl_model.predict(X_test_final)
rmse, r_squared = mean_squared_error(Y_test_final, Y_pred), r2_score(Y_test_final, Y_pred)
variance_slope, variance_intercept = coef_variance(skl_model.coef_, skl_model.intercept_, "final")
variance_rmse, variance_r_square = metrics_variance(rmse, r_squared, "final")
skl_stats = np.asarray(["SKLearn SGDRegressor", "final", 0.0001, 100, skl_model.coef_, skl_model.intercept_, r_squared, rmse, variance_slope, variance_intercept, variance_rmse, variance_r_square], dtype=object)
tests.loc[tests.shape[0]] = skl_stats

In [29]:
tests

,Class Name,Test,Learning Rate,Number of iterations,Slope,Intercept,RMSE,R-Squared,Variance_slope,Variance_intercept,Variance_RMSE,Variance_R-Squared
0,My implementation after changes,first,0.02000,100,[3.679408922981855],-0.265056,None,None,[0.12739983701814506],-0.80756,None,None
1,My implementation after changes,rmse,0.02000,4000,[7.075308146334526],-21.7909,6.50028,0.488222,None,None,-0.0480165,0.00753291
2,My implementation after changes,filter,0.02000,4000,[7.075308146334526],-21.7909,6.50028,0.488222,None,None,-0.84442,0.0637857
3,My implementation after changes,z-scale,0.02000,300,[0.6338575637915446],-0.0203856,0.718903,0.553155,None,None,-0.00123481,0.00062305
4,My implementation after changes,min-max,0.02000,300,[0.20489837081446635],0.272002,0.173954,0.187003,None,None,None,None
5,My implementation after changes,final,0.21001,1001,[0.6926164760048658],-1.68315e-16,1.27421,-0.677037,[-0.1426164760048657],8.65899e-05,-0.794207,1.16704
6,SKLearn SGDRegressor,final,0.00010,100,[0.5518054902536312],[2.4196097514628977e-05],0.496274,0.487676,[-0.0018054902536311923],[6.239375418537103e-05],-0.00767605,-0.00627422


In [30]:
tests.to_csv("report.csv")

In [31]:
from google.colab import files
files.download('report.csv') 

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>